# Why Animate

Here are some reasons you may want to use ahlive to animate your data to life!

### add dimension

Data can be mapped to color, and also time!

In [ ]:
import ahlive as ah
import xarray as xr

ds = xr.tutorial.open_dataset("air_temperature")
ds["month"] = ds["time"].dt.strftime("%m - %b")
ds = ds.groupby("month").mean()

ds_kcmi = ds["air"].sel(lon=-88.5 + 360, lat=40, method="nearest")
tmpf_inline = ds_kcmi * 1.8 - 459.67

ah.Dataset(
    ds,
    xs="lon",
    ys="lat",
    cs="air",
    state_labels="month",
    title="2013-2014 Monthly Averaged Temperatures",
    projection="LambertConformal",
    coastline=True,
    revert="boomerang",
    fps=10
).reference(
    x0s=[-88.5] * 12,  # help wanted to simplify this method!
    y0s=[40] * 12,
    inline_labels=tmpf_inline
).config(**{
    "ref_inline": dict(suffix="°F", color="black"),
    "ref_plot": dict(color="black"),
    "state": dict(color="black")
}).render()

### tell story

Present the plot, piece by piece, and highlight the important parts to tell a short story.

In [ ]:
import ahlive as ah

df = ah.open_dataset(
    label="iem_asos",
    stn=["KDEN"],
    ini="2019-10-09",
    end="2019-10-10",
    tz="mst",
    data="tmpf",
    verbose=True
).dropna(subset=["tmpf"])

ah.DataFrame(
    df,
    xs="valid",
    ys="tmpf",
    xlims="fixed",
    ylim0s="explore",
    inline_labels="tmpf",
    ylabel="°F",
    ymargins=(0.3, 0.15),
    title="2019-10-09 Temperature at Denver Airport",
).remark(
    "Sunrise at 7 AM",
    xs="2019-10-09 06:53"
).remark(
    "High reached at 3 PM",
    ys=82,
    durations=3
).remark(
    "Low reached at 11 PM, change of 52°F within 8 hours!",
    ys=30,
    first=True,
    durations=3,
).config(
    "inline",
    suffix="°F"
).render()

<div class="alert alert-warning">

Adding extra dimensionality to a visualization does not equate to better! Remember, the primary goal of a data visualization is to communicate an aspect of the data effectively.

</div>

### show progression

What better way to show change over time... than over the time dimension!

In [ ]:
import ahlive as ah

df = ah.open_dataset(
    "owid_nuclear_weapons_tests_arms_control_association_2020",
    names=["country", "year", "count"], verbose=True
)
df = df.loc[df["country"].isin(["Russia", "United States"])]

ah.DataFrame(
    df,
    xs="country",
    ys="count",
    label="country",
    title="Nuclear Arms Test",
    inline_labels="count",
    state_labels="year",
    chart="barh",
    preset="race",
    xlim1s="explore",
    figsize=(12, 5),
    frames=5
).config(**{
    "spacing": dict(left=0.2),
    "durations": dict(transition_frames=0.08)
}).render()

### emphasize extremes

By having dynamic limits, extremes can be easily highlighted.

In [ ]:
import ahlive as ah

df = ah.open_dataset(
    "owid_conflict_deaths_by_country_ucdp_2019",
    names=["country", "year", "deaths"], verbose=True
)
df = df.loc[df["country"].isin(df.nlargest(7, "deaths")["country"].unique())]

ah.DataFrame(
    df,
    xs="year",
    ys="deaths",
    label="country",
    state_labels="year",
    inline_labels="deaths",
    title="Conflict Deaths by Country",
    chart="scatter",
    preset="trail",
    join="cascade",
    ylim1s="explore",
    xlim1s="fixed",
    figsize=(10, 5),
    frames=5,
    fps=30,
).config(
    "preset", chart="both"
).render()

### engage audience

Sometimes, it's just more intriguing!

In [ ]:
import ahlive as ah
import matplotlib.pyplot as plt
from cycler import cycler

df = ah.open_dataset(
    "owid_causes_of_death_vs_media_coverage_shen_et_al_2018", verbose=True,
    names=["cause", "year", "actual", "google search", "nyt", "guardian"]
).dropna().melt(["cause", "year"], var_name="source", value_name="share")
df["source"] = df["source"].str.upper()

cmap = plt.get_cmap("plasma", df["cause"].nunique())
plt.rc("axes", prop_cycle=cycler("color", cmap.colors))

sources = df["source"].unique()
ah.layout([
    ah.DataFrame(
        df.loc[df["source"] == source],
        xs="share",
        label="cause",
        title=source,
        xlabel=" ",
        suptitle="Shen et al. (2018) Death: Reality vs Reported",
        chart="pie",
        legend=False,
        state_labels="year",
        inline_labels="cause",
        figsize=(15, 12),
        fps=5,
        autopct="%.0f%%",
        normalize=True,
    ) for source in sources
]).config(**{
    "state": dict(xy="suptitle"),
    "inline": dict(
        offset=1.25,
        color="black",
        fontsize=12,
        clip_on=False
    )
}).cols(2).render()